In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
train = pd.read_csv('/kaggle/input/EnsiasAIClub-CK/train_data.csv')
test = pd.read_csv('/kaggle/input/EnsiasAIClub-CK/test_data.csv')

In [4]:
train['Datetime'] = pd.to_datetime(train['Datetime'], format='%m/%d/%Y %H:%M')
test['Datetime'] = pd.to_datetime(test['Datetime'], format='%m/%d/%Y %H:%M')

In [5]:
def create_lag_features(df, column, window):
    for i in range(1, window+1):
        df[column + '_lag_' + str(i)] = df[column].shift(i)
    return df

def create_rolling_features(df, column, window):
    for i in range(1, window+1):
        df[column + '_rolling_mean_' + str(i)] = df[column].shift(1).rolling(i).mean()
    return df

def create_datetime_features(df):
    df['month'] = df[datetime_column].dt.month
    df['day'] = df[datetime_column].dt.day
    df['hour'] = df[datetime_column].dt.hour
    df['minute'] = df[datetime_column].dt.minute
    df['weekday'] = df[datetime_column].dt.weekday
    df['weekofyear'] = df[datetime_column].dt.weekofyear
    df['quarter'] = df[datetime_column].dt.quarter
    return df

In [6]:
windows = 20
datetime_column='Datetime'
for column in ['Temperature', 'Humidity', 'WindSpeed']:
    train = create_lag_features(train, column, windows)
    test = create_lag_features(test, column, windows)

    train = create_rolling_features(train, column, windows)
    test = create_rolling_features(test, column, windows)

# create datetime features
train = create_datetime_features(train)
test = create_datetime_features(test)
train.head()

train.fillna(method='ffill', inplace=True)
test.fillna(method='ffill', inplace=True)

train.fillna(method='bfill', inplace=True)
test.fillna(method='bfill', inplace=True)

# drop datetime column
train.drop('Datetime', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [7]:
from scipy import stats

transformed_power_data1, lambda_value1 = stats.boxcox(train["PowerConsumption_Zone1"])

train["power_consumption1"]=transformed_power_data1

In [8]:
from scipy import stats

transformed_power_data2, lambda_value2 = stats.boxcox(train["PowerConsumption_Zone2"])

train["power_consumption2"]=transformed_power_data2

In [9]:
from scipy import stats

transformed_power_data3, lambda_value3 = stats.boxcox(train["PowerConsumption_Zone3"])

train["power_consumption3"]=transformed_power_data3

In [10]:
xtrain=train.drop(["GeneralDiffuseFlows","DiffuseFlows","PowerConsumption_Zone1","PowerConsumption_Zone2","PowerConsumption_Zone3","power_consumption1","power_consumption2","power_consumption3"],axis=1)
ytrain1=train['power_consumption1']
ytrain2=train['power_consumption2']
ytrain3=train['power_consumption3']

In [11]:
from catboost import CatBoostRegressor

model1= CatBoostRegressor(learning_rate=0.4,depth=12,n_estimators=1000)
model2= CatBoostRegressor(learning_rate=0.4,depth=12,n_estimators=1000)
model3= CatBoostRegressor(learning_rate=0.4,depth=12,n_estimators=1000)

X_train1 , X_val1 , y_train1 ,y_val1 = train_test_split(xtrain,ytrain1,test_size=0.2,random_state=12)
X_train2 , X_val2 , y_train2 ,y_val2 = train_test_split(xtrain,ytrain2,test_size=0.2,random_state=12)
X_train3 , X_val3 , y_train3 ,y_val3 = train_test_split(xtrain,ytrain3,test_size=0.2,random_state=12)

model1.fit(X_train1, y_train1,eval_set=[(X_val1, y_val1)],verbose=True)
model2.fit(X_train2, y_train2,eval_set=[(X_val2, y_val2)],verbose=True)
model3.fit(X_train3, y_train3,eval_set=[(X_val3, y_val3)],verbose=True)

0:	learn: 4.7885219	test: 4.8512524	best: 4.8512524 (0)	total: 867ms	remaining: 14m 25s
1:	learn: 3.5006958	test: 3.5583065	best: 3.5583065 (1)	total: 1.64s	remaining: 13m 37s
2:	learn: 2.7954993	test: 2.8498084	best: 2.8498084 (2)	total: 2.41s	remaining: 13m 20s
3:	learn: 2.3602747	test: 2.4167346	best: 2.4167346 (3)	total: 3.36s	remaining: 13m 57s
4:	learn: 2.0517967	test: 2.0992139	best: 2.0992139 (4)	total: 4.31s	remaining: 14m 17s
5:	learn: 1.8570066	test: 1.9096989	best: 1.9096989 (5)	total: 5.07s	remaining: 13m 59s
6:	learn: 1.7431179	test: 1.8018282	best: 1.8018282 (6)	total: 5.83s	remaining: 13m 47s
7:	learn: 1.6609393	test: 1.7232053	best: 1.7232053 (7)	total: 6.59s	remaining: 13m 37s
8:	learn: 1.5562796	test: 1.6201278	best: 1.6201278 (8)	total: 7.35s	remaining: 13m 29s
9:	learn: 1.4936057	test: 1.5637430	best: 1.5637430 (9)	total: 8.14s	remaining: 13m 26s
10:	learn: 1.4211974	test: 1.4972140	best: 1.4972140 (10)	total: 8.92s	remaining: 13m 21s
11:	learn: 1.3764527	test: 1.4

In [12]:
from scipy.special import inv_boxcox
test.drop('Datetime', axis=1, inplace=True)
train.drop(["GeneralDiffuseFlows","DiffuseFlows"],axis=1)
y_pred1=model1.predict(test)
y_pred1 = inv_boxcox(y_pred1, lambda_value1)

y_pred2=model2.predict(test)
y_pred2 = inv_boxcox(y_pred2, lambda_value2)

y_pred3=model3.predict(test)
y_pred3 = inv_boxcox(y_pred3, lambda_value3)

In [13]:
sample=pd.read_csv('/kaggle/input/EnsiasAIClub-CK/sample_data.csv')
sample["predicted_Zone1"]= y_pred1
sample["predicted_Zone2"]= y_pred2
sample["predicted_Zone3"]= y_pred3
sample.head()

,Datetime,predicted_Zone1,predicted_Zone2,predicted_Zone3
0,2/12/2017 14:40,32376.451325,18399.960514,17819.549633
1,11/22/2017 15:30,30254.546328,25078.172529,10570.841804
2,10/15/2017 1:50,27756.743485,16619.108900,12066.613996
3,9/11/2017 23:00,36320.645833,21601.456679,17600.119437
4,12/26/2017 1:20,23139.172477,18581.143268,10284.017814


In [14]:
sample.to_csv('finalsubmission.csv', index=False)